#### Análise de tweets: Desastre ou não?
- Este é um projeto de ciência de dados de ponta a ponta (da coleta de dados ao deploy), que tem como objetivo <b>analisar</b> um conjunto de <b>tweets</b> e <b>prever a probabilidade de um tweet estar relacionado a um desastre real.</b> Portanto, trata-se de um problema de classificação binária (aprendizado supervisionado, conjunto de dados rotulado) no qual a variável dependente é 1 para o caso de tweets relacionados a desastres reais e 0 para tweets não relacionados.
- O pipeline de solução, com base no framework <B>CRISP-DM</b>, consiste dos seguintes <b>passos:</b><br>

    <b>0. Entendimento do problema de negócio.</b><br>
    <b>1. Entendimento dos dados.</b><br>
    <b>2. Preparação dos dados.</b><br>
    <b>3. Modelagem.</b><br>
    <b>4. Avaliação.</b><br>
    <b>5. Deploy/Implantação.</b><br>

- Neste notebook, será realizada a <b>modelagem para a predição da probabilidade de um tweet estar relacionado a um desastre</b>, cobrindo os passos 3 a 5 do framework supracitado. O objetivo da modelagem consiste em <b>construir um modelo</b> capaz de <b>prever essa chance acuradamente</b> e criar uma <b>Flask API</b>, de forma que seja possível prever a probabilidade de um tweet estar associado a um desastre fornecendo-o em texto.
- Aqui, adotaremos a abordagem de <b>prever a probabilidade</b>, levando em consideração a <b>informação de confiança.</b> Isso nos permite avaliar quão provável é que um determinado tweet represente uma catástrofe real. Tal abordagem facilita a gestão de risco por parte das agências mencionadas no problema de negócio em 'eda.ipynb', possibilitando a <b>priorização da atenção</b> para tweets com maior probabilidade. Além disso, ><b>reduz a propagação de notícias falsas</b>, uma vez que rótulos binários poderiam classificar erroneamente vários tweets, especialmente ao utilizar pontos de corte mais baixos para a classificação (trade-off precision-recall).
- Nesse sentido, <B>métricas</b> como ROC-AUC, PR-AUC, Brier Score e KS serão <b>priorizadas.</b> Entretanto, olharemos para diversas outras. Mesmo adotando o critério probabilístico, é interessante obter um bom recall. O recall pode ser interpretado como o alcance, e de fato, é melhor que "alcancemos" o maior número de desastres reais possível, isto é, que o modelo seja capaz de identificar uma grande parte dos tweets de fato associados a desastres.
- O pipeline de modelagem consiste em:
- 0. Limpeza e feature engineering.
- 1. Split dos dados em treino e teste, estratificado.
- 2. Pré-processamento de dados.
- 3. Comparação de diversos modelos através de validação cruzada k-fold estratificada.
- 4. Tunagem de hiperparâmetros do modelo com class_weight e otimização bayesiana.
- 5. Avaliação final no conjunto de testes.
- 6. Deploy.
- Cada etapa e cada decisão tomada é abordada em detalhes abaixo. Os insights obtidos na eda guiarão as decisões daqui para frente.

Importando as bibliotecas novamente.

In [22]:
# Data manipulation and visualization.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# Cleaning and preparation.
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from spellchecker import SpellChecker
from collections import Counter
from textblob import TextBlob

# Utils.
from src.modelling_utils import *

# Definições de cores -> todas estão numa escala de mais escura para mais clara.
CINZA1, CINZA2, CINZA3 = '#231F20', '#414040', '#555655'
CINZA4, CINZA5, CINZA6 = '#646369', '#76787B', '#828282'
CINZA7, CINZA8, CINZA9 = '#929497', '#A6A6A5', '#BFBEBE'
AZUL1, AZUL2, AZUL3, AZUL4 = '#174A7E', '#4A81BF', '#94B2D7', '#94AFC5'
VERMELHO1, VERMELHO2, VERMELHO3, VERMELHO4, VERMELHO5 = '#DB0527', '#E23652', '#ED8293', '#F4B4BE', '#FBE6E9'
VERDE1, VERDE2 = '#0C8040', '#9ABB59'
LARANJA1 = '#F79747'
AMARELO1, AMARELO2, AMARELO3, AMARELO4, AMARELO5 = '#FFC700', '#FFCC19', '#FFEB51', '#FFE37F', '#FFEEB2'
BRANCO = '#FFFFFF'

# Visualize all the columns.
pd.set_option('display.max_columns', None)


# Filter warnings.
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\peedr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\peedr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\peedr\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\peedr\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\peedr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\peedr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_d

Coletando novamente os dados.

In [2]:
# Reading the data.
train_path = '../input/train.csv'
test_path = '../input/test.csv'
sample_path = '../input/sample_submission.csv'
df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
sample_df = pd.read_csv(sample_path)

Rápida visualização e informação.

In [3]:
df.head()

,id,keyword,location,text,target
0,1118,blew%20up,"Brooklyn, NY",@YahooSchwab easy way to look good after the R...,0
1,1558,bomb,NaN,@dopeitsval ahh you're bomb baby ??,0
2,4830,evacuation,NaN,Run out evacuation hospital indexing remedial ...,1
3,3674,destroy,Trackside California,Wow Crackdown 3 uses multiple servers in multi...,0
4,5368,fire%20truck,NaN,wild night in the village of pugwash every fir...,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [5]:
print(f'O dataset possui {df.shape[0]} linhas e {df.shape[1]} colunas.')

O dataset possui 7613 linhas e 5 colunas.


Valores nulos e duplicados.

In [6]:
isna_df = df.isna().sum().to_frame().rename(columns={0: 'count'})
isna_df['pct'] = isna_df['count'] / len(df) * 100
isna_df.sort_values(by=['pct'], ascending=False)

,count,pct
location,2533,33.272035
keyword,61,0.801261
id,0,0.000000
text,0,0.000000
target,0,0.000000


In [7]:
print(f'Observações duplicadas: {df.duplicated().sum()}.')
print(f'Ids duplicados: {df.id.duplicated().sum()}')

Observações duplicadas: 0.
Ids duplicados: 0


#### 0. Limpeza
- Antes dos split, é necessário limpar os dados, aplicando as mesmas transformações realizadas na eda.
- 1. Remoção de links, tags html, @s de usuários, pontuações, caracteres especiais, números e stopwords (palavras sem natureza informativa) com o objetivo de não prejudicar a semântica, não introduzir ruído, e diminuir a dimensionalidade. Além disso, há uma questão ética em não usar o nome dos usuários. Tudo isso feito através de expressões regulares, que são padrões de texto que permitem realizar buscas complexas e operações de manipulação em strings. Um cheat sheet de referência será usado: https://www.datacamp.com/cheat-sheet/regular-expresso.
- 2. Conversão das palavras a lowercase para padronizá-las e reduzir a dimensionalidade.
- 3. Lematização, para manter a coerência linguística das palavras ao reduzi-las a suas raízes ou lemas. Utiliza-se part of speech tags aqui. Stemming não é aplicado pois, apesar de a lematização ser mais custosa, desejo demonstrar mais conhecimento e técnica. Além disso, no deploy, processaremos um único tweet por vez, não havendo gargalos significativos.
- 4. Para a modelagem, estamos interessados apenas na coluna contendo os tweets como variável independente, portanto, essa será separada das outras.
- Estes passos foram encapsulados em uma função 'clean_text' em 'modelling_utils.py', arquivo que contém funções úteis para a modelagem. 

In [19]:
# Separando apenas as colunas text e target para a modelagem.
clean_df = df[['text', 'target']].copy()
clean_df.head()

,text,target
0,@YahooSchwab easy way to look good after the R...,0
1,@dopeitsval ahh you're bomb baby ??,0
2,Run out evacuation hospital indexing remedial ...,1
3,Wow Crackdown 3 uses multiple servers in multi...,0
4,wild night in the village of pugwash every fir...,1


In [21]:
# Acessando o resultado.
clean_df = nlp_data_cleaning(clean_df)
clean_df['clean_text'].head(15)

0           easy way look good ray rice fiascothat blow
1                                   ahh youre bomb baby
2     run evacuation hospital index remedial angiopl...
3     wow crackdown us multiple server multiplayer u...
4     wild night village pugwash every fire truck to...
5                                bed look like war zone
6               fund need rescue abandon cocker spaniel
7            burn building mean like burnt black church
8                                               survive
9     emergency drink water plan download guide engl...
10    california fire rage forest service sound alar...
11               something place prevent skynet perhaps
12           new smp ignition knock detonation sensor k
13    memorial unveil travis county deputy kill sept...
14    live balance life balance fear # allah hope me...
Name: clean_text, dtype: object